# Trial segmentation script

This notebook is intended for the breakdown, classification and analysis of the behavioural states shown in the video, starting frmo a trial segmentation. 

- `trial segmentation` takes the output of simplerCode.py (video + .csv), and segments the video based on mouse entry and exit times, excluding when the mouse entry/exit was not detected. 
- we then will manually sort the trials into `exploitative`, `explorative` and `nest`
- extract speed and trajectories per trial
- identify behavioural syllables with keypoint moseq 2D



In [4]:
# install libraries
# !pip install -q pandas moviepy imageio-ffmpeg

In [5]:
# Get a reliable ffmpeg binary (bundled by imageio-ffmpeg) and verify it
%pip install -q imageio-ffmpeg pandas

import os, shutil, subprocess, imageio_ffmpeg
from pathlib import Path

# Save the path and sanitize (strip any accidental quotes/whitespace)
os.environ["IMAGEIO_FFMPEG_EXE"] = imageio_ffmpeg.get_ffmpeg_exe()
FFMPEG = os.environ["IMAGEIO_FFMPEG_EXE"].strip().strip('"')

print("ffmpeg binary:", repr(FFMPEG))
print("Path exists? ", Path(FFMPEG).exists())

# This must print ffmpeg version and NOT raise
subprocess.run([FFMPEG, "-version"], check=True)

Note: you may need to restart the kernel to use updated packages.
ffmpeg binary: 'c:\\Users\\shahd\\OneDrive\\Documents\\GitHub\\mice-maze\\.venv\\lib\\site-packages\\imageio_ffmpeg\\binaries\\ffmpeg-win-x86_64-v7.1.exe'
Path exists?  True


CompletedProcess(args=['c:\\Users\\shahd\\OneDrive\\Documents\\GitHub\\mice-maze\\.venv\\lib\\site-packages\\imageio_ffmpeg\\binaries\\ffmpeg-win-x86_64-v7.1.exe', '-version'], returncode=0)

In [6]:
import os
import pandas as pd
from pathlib import Path
import subprocess, shutil, math

import re

Here we just isolate `mouse 6357` and `mouse 6359`, sessions `1.1`, `3.5` and `3.6`

In [7]:
main_dir = "C:/Users/shahd/Box/Awake Project/Maze data/simplermaze/"

#this is where the csv files of interest are stored
csvs = []

#this is where we store the videos
videos = []


for i in os.listdir(main_dir):
    if "6357" in i or "6359" in i:
        mouse_dir = os.path.join(main_dir, i)
        # print(mouse_dir[-10:])
        for session in os.listdir(mouse_dir):
            if 'habituation' in session or '1.1' in session or "3.5" in session or "3.6" in session or "3.7" in session or "3.8" in session:
                session_dir = os.path.join(mouse_dir, session)
                # print(f"    -{session_dir[-10:]}")

                for file in os.listdir(session_dir):

                    full_path_file = os.path.join(session_dir, file)

                    if "trial_info.csv" in file and "clean" not in file:
                        csvs.append(full_path_file)
                        # print(f"        -{file}")
                    elif "mp4" in file:
                        videos.append(full_path_file)

csv_video = dict(zip(csvs, videos))


In [8]:

for key,value in csv_video.items():
    n_key = os.path.basename(key)
    n_value = os.path.basename(value)
    print(n_key + "    :     "+ n_value)

mouse6357_session3.5_trial_info.csv    :     6357_2024-08-27_13_05_19s3.5.mp4
mouse6357_session3.6_trial_info.csv    :     6357_2024-08-28_11_58_14s3.6.mp4
mouse6357_session3.7_trial_info.csv    :     6357_2024-08-29_10_23_02s3.7.mp4
mouse6357_session3.8_trial_info.csv    :     6357_2024-08-30_10_07_55s3.8.mp4
mouse6357_session1.1_trial_info.csv    :     6357_2024-08-15_11_23_10.mp4
mouse6359_session3.5_trial_info.csv    :     6359_2024-08-27_14_08_35s3.5.mp4
mouse6359_session3.6_trial_info.csv    :     6359_2024-08-28_13_28_27s3.6.mp4
mouse6359_session3.7_trial_info.csv    :     6359_2024-08-29_11_39_28s3.7.mp4
mouse6359_session3.8_trial_info.csv    :     6359_2024-08-30_11_28_10s3.8.mp4
mouse6359_session1.1_trial_info.csv    :     6359_2024-08-15_14_05_08.mp4


In [9]:
# def ffmpeg_bin():
#     return os.environ.get("IMAGEIO_FFMPEG_EXE") or "ffmpeg"

# def ms_to_sec(x): 
#     return None if pd.isna(x) else float(x)/1000.0

# def sanitize(s: str) -> str:
#     for ch in '<>:"/\\|?*': s = s.replace(ch, "_")
#     return s

# def build_output_name(base_label: str, trial_idx: int, ext=".mp4"):
#     return f"{sanitize(base_label)}_trial_{int(trial_idx):03d}{ext}"

# def cut_with_ffmpeg(input_video: Path, start_s: float, end_s: float, out_path: Path, reencode: bool):
#     dur = max(0.0, end_s - start_s)
#     out_path.parent.mkdir(parents=True, exist_ok=True)
#     if reencode:
#         cmd = [
#             ffmpeg_bin(), "-hide_banner", "-loglevel", "error",
#             "-ss", f"{start_s:.3f}", "-t", f"{dur:.3f}",
#             "-i", str(input_video),
#             "-map", "0:v:0?", "-c:v", "libx264", "-preset", "veryfast", "-crf", "18",
#             "-movflags", "+faststart", "-reset_timestamps", "1",
#             str(out_path)
#         ]
#     else:
#         cmd = [
#             ffmpeg_bin(), "-hide_banner", "-loglevel", "error",
#             "-ss", f"{start_s:.3f}", "-to", f"{end_s:.3f}",
#             "-i", str(input_video),
#             "-map", "0:v:0?", "-c", "copy",
#             "-movflags", "+faststart", "-reset_timestamps", "1",
#             str(out_path)
#         ]
#     subprocess.run(cmd, check=True)

# def segment_one_session(
#     video, trials_csv, outdir=None, base_label=None,
#     offset_ms=0.0, padding_ms=0.0, method="copy", inplace=False,
#     column_name="video_segment_path"
# ):
#     video = Path(video).resolve()
#     trials_csv = Path(trials_csv).resolve()
#     outdir = Path(outdir).resolve() if outdir else (video.parent / "segments")
#     outdir.mkdir(parents=True, exist_ok=True)

#     df = pd.read_csv(trials_csv)
#     for col in ["mouse_enter_time","end_trial_time"]:
#         if col not in df.columns:
#             raise ValueError(f"Required column '{col}' missing in {trials_csv.name}")

#     trial_ids = df["trial_ID"].tolist() if "trial_ID" in df.columns else list(df.index)
#     if column_name not in df.columns:
#         df[column_name] = pd.NA

#     base_label = base_label if (base_label is not None and base_label != "") else video.stem

#     made = 0
#     skipped = 0
#     for i, trial_id in enumerate(trial_ids):
#         row = df.iloc[i]
#         enter_ms = row["mouse_enter_time"]; exit_ms = row["end_trial_time"]
#         if pd.isna(enter_ms) or pd.isna(exit_ms):
#             skipped += 1; continue

#         start_ms = max(0.0, float(enter_ms) + float(offset_ms) - float(padding_ms))
#         end_ms   = max(0.0, float(exit_ms)  + float(offset_ms) + float(padding_ms))
#         if end_ms <= start_ms:
#             skipped += 1; continue

#         start_s, end_s = ms_to_sec(start_ms), ms_to_sec(end_ms)
#         out_path = outdir / build_output_name(base_label, trial_id)
#         try:
#             cut_with_ffmpeg(video, start_s, end_s, out_path, reencode=(method=="reencode"))
#             df.at[i, column_name] = str(out_path)
#             made += 1
#         except Exception as e:
#             print(f"[WARN] {video.name} | trial {trial_id}: {e}")
#             skipped += 1

#     updated = trials_csv if inplace else trials_csv.with_name(trials_csv.stem + "_with_segments.csv")
#     df.to_csv(updated, index=False)
#     return {"segments_made": made, "trials_skipped": skipped, "updated_csv": str(updated)}

In [10]:
# --- FFMPEG-ONLY HELPERS (uses the verified binary) ---
from pathlib import Path
import os, subprocess, pandas as pd, shutil

# Keep using the verified binary from imageio-ffmpeg:
# (you already set IMAGEIO_FFMPEG_EXE; this will pick it up)
def ffmpeg_bin():
    prog = os.environ.get("IMAGEIO_FFMPEG_EXE") or shutil.which("ffmpeg") or "ffmpeg"
    return prog.strip().strip('"')

def preflight():
    prog = ffmpeg_bin()
    if not (Path(prog).exists() or shutil.which(prog)):
        raise RuntimeError(f"ffmpeg not found at '{prog}'")
    subprocess.run([prog, "-version"], check=True)

def ms_to_sec(x): 
    return None if pd.isna(x) else float(x)/1000.0

def sanitize(s: str) -> str:
    for ch in '<>:"/\\|?*': s = s.replace(ch, "_")
    return s

def build_output_name(base_label: str, trial_idx: int, ext=".mp4"):
    return f"{sanitize(base_label)}_trial_{int(trial_idx):03d}{ext}"

def cut_with_ffmpeg(input_video: Path, start_s: float, end_s: float, out_path: Path, reencode: bool):
    prog = ffmpeg_bin()
    dur = max(0.0, end_s - start_s)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    cmd = (
        [prog,"-hide_banner","-loglevel","error","-ss",f"{start_s:.3f}","-t",f"{dur:.3f}",
         "-i",str(input_video),"-map","0:v:0?","-c:v","libx264","-preset","veryfast","-crf","18",
         "-movflags","+faststart","-reset_timestamps","1",str(out_path)]
        if reencode else
        [prog,"-hide_banner","-loglevel","error","-ss",f"{start_s:.3f}","-to",f"{end_s:.3f}",
         "-i",str(input_video),"-map","0:v:0?","-c","copy","-movflags","+faststart",
         "-reset_timestamps","1",str(out_path)]
    )
    subprocess.run(cmd, check=True)

def segment_one_session(
    video, trials_csv, outdir=None, base_label=None,
    offset_ms=0.0, padding_ms=0.0, method="copy", inplace=False,
    column_name="video_segment_path"
):
    preflight()

    video = Path(video).expanduser().resolve()
    trials_csv = Path(trials_csv).expanduser().resolve()
    outdir = Path(outdir).expanduser().resolve() if outdir else (video.parent / "segments")

    if not video.exists():
        raise FileNotFoundError(f"Video not found: {video}")
    if not trials_csv.exists():
        raise FileNotFoundError(f"CSV not found: {trials_csv}")

    outdir.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(trials_csv)
    for col in ["mouse_enter_time","end_trial_time"]:
        if col not in df.columns:
            raise ValueError(f"Required column '{col}' missing in {trials_csv.name}")

    trial_ids = df["trial_ID"].tolist() if "trial_ID" in df.columns else list(df.index)
    if column_name not in df.columns:
        df[column_name] = pd.NA

    base_label = base_label if (base_label is not None and base_label != "") else video.stem

    made = skipped = 0
    for i, trial_id in enumerate(trial_ids):
        row = df.iloc[i]
        enter_ms = row["mouse_enter_time"]; exit_ms = row["end_trial_time"]
        if pd.isna(enter_ms) or pd.isna(exit_ms):
            skipped += 1; continue

        start_ms = max(0.0, float(enter_ms) + float(offset_ms) - float(padding_ms))
        end_ms   = max(0.0, float(exit_ms)  + float(offset_ms) + float(padding_ms))
        if end_ms <= start_ms:
            skipped += 1; continue

        start_s, end_s = ms_to_sec(start_ms), ms_to_sec(end_ms)
        out_path = outdir / build_output_name(base_label, trial_id)

        try:
            cut_with_ffmpeg(video, start_s, end_s, out_path, reencode=(method=="reencode"))
            df.at[i, column_name] = str(out_path)
            made += 1
        except subprocess.CalledProcessError as e:
            # ffmpeg ran but failed (bad input?), show helpful paths:
            print(f"[WARN] ffmpeg failed for trial {trial_id}")
            print("  video :", video, "| exists:", video.exists())
            print("  out   :", out_path)
            skipped += 1

    updated = trials_csv if inplace else trials_csv.with_name(trials_csv.stem + "_with_segments.csv")
    df.to_csv(updated, index=False)
    return {"segments_made": made, "trials_skipped": skipped, "updated_csv": str(updated)}
# --- end helpers ---


In [11]:
# from pathlib import Path

# one_csv, one_video = next(iter(csv_video.items()))
# one_csv  = str(Path(one_csv).expanduser().resolve())
# one_video = str(Path(one_video).expanduser().resolve())

# print("CSV  :", one_csv,  "| exists:", Path(one_csv).exists())
# print("VIDEO:", one_video, "| exists:", Path(one_video).exists())

# res = segment_one_session(
#     video=one_video,
#     trials_csv=one_csv,
#     outdir=Path(one_video).parent / "segments",
#     method="reencode",         # precise boundaries
#     inplace=False,
#     column_name="video_segment_path",
# )
# res


In [22]:

summaries = []
for trials_csv, video in csv_video.items():
    trials_csv = str(Path(trials_csv).expanduser().resolve())
    video = str(Path(video).expanduser().resolve())
    outdir = Path(video).parent / "segments"
    res = segment_one_session(
        video=video,
        trials_csv=trials_csv,
        outdir=outdir,
        method="copy",
        # inplace=False
        column_name="video_segment_path",
    )
    summaries.append({"video": video, "trials_csv": trials_csv, **res})

pd.DataFrame(summaries)

Done 6357_2024-08-27_13_05_19s3.5.mp4: made=53, skipped=4, elapsed=5.1s
[TIMEOUT] 6357_2024-08-28_11_58_14s3.6.mp4 | trial 0 took >120s; skipping
Done 6357_2024-08-28_11_58_14s3.6.mp4: made=55, skipped=12, elapsed=199.5s
[TIMEOUT] 6357_2024-08-29_10_23_02s3.7.mp4 | trial 0 took >120s; skipping
Done 6357_2024-08-29_10_23_02s3.7.mp4: made=66, skipped=3, elapsed=155.0s
[TIMEOUT] 6357_2024-08-30_10_07_55s3.8.mp4 | trial 0 took >120s; skipping
Done 6357_2024-08-30_10_07_55s3.8.mp4: made=57, skipped=16, elapsed=138.7s
[TIMEOUT] 6357_2024-08-15_11_23_10.mp4 | trial 0 took >120s; skipping
Done 6357_2024-08-15_11_23_10.mp4: made=37, skipped=13, elapsed=189.8s
[TIMEOUT] 6359_2024-08-27_14_08_35s3.5.mp4 | trial 0 took >120s; skipping
Done 6359_2024-08-27_14_08_35s3.5.mp4: made=52, skipped=12, elapsed=135.5s
[TIMEOUT] 6359_2024-08-28_13_28_27s3.6.mp4 | trial 0 took >120s; skipping
Done 6359_2024-08-28_13_28_27s3.6.mp4: made=73, skipped=23, elapsed=145.2s
[TIMEOUT] 6359_2024-08-29_11_39_28s3.7.mp4 

,video,trials_csv,segments_made,trials_skipped,updated_csv
0,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,53,4,C:\Users\shahd\Box\Awake Project\Maze data\sim...
1,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,55,12,C:\Users\shahd\Box\Awake Project\Maze data\sim...
2,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,66,3,C:\Users\shahd\Box\Awake Project\Maze data\sim...
3,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,57,16,C:\Users\shahd\Box\Awake Project\Maze data\sim...
4,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,37,13,C:\Users\shahd\Box\Awake Project\Maze data\sim...
5,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,52,12,C:\Users\shahd\Box\Awake Project\Maze data\sim...
6,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,73,23,C:\Users\shahd\Box\Awake Project\Maze data\sim...
7,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,61,8,C:\Users\shahd\Box\Awake Project\Maze data\sim...
8,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,57,14,C:\Users\shahd\Box\Awake Project\Maze data\sim...
9,C:\Users\shahd\Box\Awake Project\Maze data\sim...,C:\Users\shahd\Box\Awake Project\Maze data\sim...,40,1,C:\Users\shahd\Box\Awake Project\Maze data\sim...


NameError: name 'pd' is not defined

taking too long, adding some checks to see if it works

In [13]:
from pathlib import Path
import os, subprocess, pandas as pd, shutil, time

# Use the verified binary you already printed
def ffmpeg_bin():
    prog = os.environ.get("IMAGEIO_FFMPEG_EXE") or shutil.which("ffmpeg") or "ffmpeg"
    return prog.strip().strip('"')

def preflight():
    prog = ffmpeg_bin()
    if not (Path(prog).exists() or shutil.which(prog)):
        raise RuntimeError(f"ffmpeg not found at '{prog}'")
    subprocess.run([prog, "-version"], check=True)

def ms_to_sec(x): 
    return None if pd.isna(x) else float(x)/1000.0

def sanitize(s: str) -> str:
    for ch in '<>:"/\\|?*': s = s.replace(ch, "_")
    return s

def build_output_name(base_label: str, trial_idx: int, ext=".mp4"):
    return f"{sanitize(base_label)}_trial_{int(trial_idx):03d}{ext}"

def cut_with_ffmpeg(input_video: Path, start_s: float, end_s: float, out_path: Path,
                    reencode: bool, timeout_sec: int = 120):
    prog = ffmpeg_bin()
    dur = max(0.0, end_s - start_s)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    # -y = auto overwrite; -nostdin = never wait for keyboard input
    cmd = (
        [prog,"-nostdin","-y","-hide_banner","-loglevel","error",
         "-ss",f"{start_s:.3f}","-t",f"{dur:.3f}",
         "-i",str(input_video),
         "-map","0:v:0?","-c:v","libx264","-preset","veryfast","-crf","18",
         "-movflags","+faststart","-reset_timestamps","1",str(out_path)]
        if reencode else
        [prog,"-nostdin","-y","-hide_banner","-loglevel","error",
         "-ss",f"{start_s:.3f}","-to",f"{end_s:.3f}",
         "-i",str(input_video),
         "-map","0:v:0?","-c","copy",
         "-movflags","+faststart","-reset_timestamps","1",str(out_path)]
    )
    subprocess.run(cmd, check=True, timeout=timeout_sec)

def segment_one_session(
    video, trials_csv, outdir=None, base_label=None,
    offset_ms=0.0, padding_ms=0.0, method="copy", inplace=False,
    column_name="video_segment_path", skip_existing=True,
    only_first_n=None, timeout_sec=120
):
    preflight()
    video = Path(video).expanduser().resolve()
    trials_csv = Path(trials_csv).expanduser().resolve()
    outdir = Path(outdir).expanduser().resolve() if outdir else (video.parent / "segments")

    if not video.exists(): raise FileNotFoundError(f"Video not found: {video}")
    if not trials_csv.exists(): raise FileNotFoundError(f"CSV not found: {trials_csv}")
    outdir.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(trials_csv)
    for col in ["mouse_enter_time","end_trial_time"]:
        if col not in df.columns:
            raise ValueError(f"Required column '{col}' missing in {trials_csv.name}")

    trial_ids = df["trial_ID"].tolist() if "trial_ID" in df.columns else list(df.index)
    if column_name not in df.columns:
        df[column_name] = pd.NA
    base_label = base_label if (base_label is not None and base_label != "") else video.stem

    made = skipped = 0
    t0 = time.time()
    for i, trial_id in enumerate(trial_ids):
        if only_first_n is not None and (made + skipped) >= only_first_n:
            break

        row = df.iloc[i]
        enter_ms = row["mouse_enter_time"]; exit_ms = row["end_trial_time"]
        if pd.isna(enter_ms) or pd.isna(exit_ms) or float(exit_ms) <= float(enter_ms):
            skipped += 1; continue

        start_ms = max(0.0, float(enter_ms) + float(offset_ms) - float(padding_ms))
        end_ms   = max(0.0, float(exit_ms)  + float(offset_ms) + float(padding_ms))
        if end_ms <= start_ms: skipped += 1; continue

        start_s, end_s = ms_to_sec(start_ms), ms_to_sec(end_ms)
        out_path = outdir / build_output_name(base_label, trial_id)

        if skip_existing and out_path.exists():
            df.at[i, column_name] = str(out_path)
            skipped += 1
            continue

        try:
            cut_with_ffmpeg(video, start_s, end_s, out_path, reencode=(method=="reencode"), timeout_sec=timeout_sec)
            df.at[i, column_name] = str(out_path)
            made += 1
        except subprocess.TimeoutExpired:
            print(f"[TIMEOUT] {video.name} | trial {trial_id} took >{timeout_sec}s; skipping")
            skipped += 1
        except subprocess.CalledProcessError as e:
            print(f"[WARN] ffmpeg failed for trial {trial_id}: {e}")
            skipped += 1

    updated = trials_csv if inplace else trials_csv.with_name(trials_csv.stem + "_with_segments.csv")
    df.to_csv(updated, index=False)
    print(f"Done {video.name}: made={made}, skipped={skipped}, elapsed={time.time()-t0:.1f}s")
    return {"segments_made": made, "trials_skipped": skipped, "updated_csv": str(updated)}


In [14]:
one_csv, one_video = next(iter(csv_video.items()))
res = segment_one_session(
    video=one_video, trials_csv=one_csv,
    outdir=Path(one_video).parent / "segments",
    method="copy", only_first_n=3, skip_existing=True, timeout_sec=60
)
res


Done 6357_2024-08-27_13_05_19s3.5.mp4: made=1, skipped=2, elapsed=0.2s


{'segments_made': 1,
 'trials_skipped': 2,
 'updated_csv': 'C:\\Users\\shahd\\Box\\Awake Project\\Maze data\\simplermaze\\mouse 6357\\2024-08-27_13_05_196357session3.5\\mouse6357_session3.5_trial_info_with_segments.csv'}

In [21]:
import pandas as pd

csv_out = r"C:/Users/shahd/Box/Awake Project/Maze data/simplermaze/mouse 6357/2024-08-27_13_05_196357session3.5/mouse6357_session3.5_trial_info_with_segments.csv"
df = pd.read_csv(csv_out)

# quick look at the first 8 trials
cols = ["mouse_enter_time","end_trial_time","video_segment_path"]
print(df[cols].head(8))

# which rows are invalid for cutting?
bad = df[df["mouse_enter_time"].isna() | df["end_trial_time"].isna() | (df["end_trial_time"] <= df["mouse_enter_time"])]
print("/nBad rows (missing/negative duration):")
print(bad[["mouse_enter_time","end_trial_time"]])

# sanity check durations
df["dur_ms"] = df["end_trial_time"] - df["mouse_enter_time"]
print("\nDuration (ms) summary:\n", df["dur_ms"].describe())


   mouse_enter_time  end_trial_time  \
0           22268.0           35060   
1           35826.0           90493   
2               NaN          125483   
3          151699.0          155543   
4          163389.0          167113   
5          179969.0          182683   
6          204891.0          208255   
7          224706.0          254501   

                                  video_segment_path  
0  C:\Users\shahd\Box\Awake Project\Maze data\sim...  
1  C:\Users\shahd\Box\Awake Project\Maze data\sim...  
2                                                NaN  
3                                                NaN  
4                                                NaN  
5                                                NaN  
6                                                NaN  
7                                                NaN  
/nBad rows (missing/negative duration):
    mouse_enter_time  end_trial_time
2                NaN          125483
36               NaN         1900568

D